# 3.3.2 Word2Vec

In [48]:
# load data 
import pandas as pd
df = pd.read_csv('clinic.csv')
df.head()


,Sent1,Sent2,Score
0,Insulin NPH Human [NOVOLIN N] 100 unit / mL su...,Insulin NPH Human [NOVOLIN N] 100 unit / mL su...,3.50
1,"Patient arrives ambulatory, Gait steady, Hist...","Complex assessment performed, Patient arrives...",2.50
2,"Peripheral IV site, established in the right f...","Peripheral IV site, present prior to arrival, ...",3.45
3,No : new confusion or inability to stay aler...,No : new confusion or inability to stay aler...,4.00
4,Spent 15 minutes with the patient and greater ...,"Nurse visit ten minutes, over half of which w...",3.00


In [49]:
# covert to list
sent1 = df['Sent1'].tolist()
sent2 = df['Sent2'].tolist()
lables = df['Score'].tolist()

In [50]:
%store -r cleaned1
%store -r cleaned2
%store -r dtree1
%store -r dtree2
%store -r feat

# data split.
train_text1 = dtree1[:750]
train_text2 = dtree2[:750]
train_labels =lables[:750]

test_text1 = dtree1[750:]
test_text2 = dtree2[750:]
test_labels =lables[750:]


In [51]:
import gensim
import gensim.downloader as gensim_api

from gensim.test.utils import datapath
from gensim.models import KeyedVectors
path_of_downloaded_bin = "/Users/aswath/PycharmProjects/mfac038/IndividualProject/PubMed-and-PMC-w2v.bin" #https://bio.nlplab.org
word_vectors = KeyedVectors.load_word2vec_format(datapath(path_of_downloaded_bin), binary=True)


In [52]:
from scipy import spatial
index2word_set = set(word_vectors.wv.index2word)


In [53]:
import numpy as np
from scipy import spatial

index2word_set = set(word_vectors.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec


In [54]:
# calculating average feature vector for each sentences
#train data
cc1=[]
cc2=[]
for i in range(0,750):
    s1_afv = avg_feature_vector(train_text1[i], model=word_vectors, num_features=200, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(train_text2[i], model=word_vectors, num_features=200, index2word_set=index2word_set)
    cc1.append(s1_afv)
    cc2.append(s2_afv)
print(len(cc1))
print(cc1[1])
print(len(cc2))
print(cc2[1])

#test data
cc3=[]
cc4=[]
for i in range(0,318):
    s3_afv = avg_feature_vector(test_text1[i], model=word_vectors, num_features=200, index2word_set=index2word_set)
    s4_afv = avg_feature_vector(test_text2[i], model=word_vectors, num_features=200, index2word_set=index2word_set)
    cc3.append(s3_afv)
    cc4.append(s4_afv)
print(len(cc3))
print(cc3[1])
print(len(cc4))
print(cc4[1])


750
[-0.20914403  0.0189739   0.10916398  0.07685421 -0.07910736  0.06374723
 -0.08940926 -0.04505254 -0.05449844  0.05254232  0.01133474 -0.0721116
  0.07052402 -0.02098625 -0.01864529 -0.1379583  -0.03052476 -0.11118361
 -0.02351088 -0.0281716   0.23679136 -0.03156919  0.01149724 -0.04644114
  0.05478407  0.12899642  0.02571652 -0.10900252 -0.04852561  0.14347619
 -0.14356391 -0.19431353  0.05600974 -0.11769733  0.09506132 -0.02353107
  0.06034361  0.0830548  -0.12045224 -0.0805388   0.19253074  0.01503594
  0.09803904  0.0372519   0.16204132 -0.13732833 -0.1229924  -0.05643224
 -0.13570508 -0.19836025  0.08172584  0.01232461  0.02102929  0.21031958
 -0.08317956 -0.0038079  -0.08657803 -0.01101736  0.10880186 -0.11988424
  0.06248564  0.04129849  0.08187519  0.07075413 -0.00206357  0.02347714
 -0.10365088  0.02387581  0.08973619  0.01554786 -0.07115928 -0.14225589
 -0.01506053  0.15825406  0.04017153  0.08326517 -0.02810805 -0.02952631
 -0.06495576  0.00989895  0.06842037 -0.06196987

In [55]:
#word2vec + Random Forest
from sklearn.ensemble import RandomForestRegressor
reg1 = RandomForestRegressor(max_depth=6).fit(cc1, train_labels)
reg2 = RandomForestRegressor(max_depth=6).fit(cc2, train_labels)
reg1
reg2

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [56]:
from sklearn.metrics import mean_squared_error
# test model
test_pred1 = reg1.predict(cc3)
test_pred2 = reg2.predict(cc4)

b_mse1 = mean_squared_error(test_labels, test_pred1)
b_mse2 = mean_squared_error(test_labels, test_pred2)

print('MSE for Sentence  1: ', b_mse1)

print("--------------------------------------------")

print('MSE for Sentence  2: ', b_mse2)



MSE for Sentence  1:  1.1226993297669186
--------------------------------------------
MSE for Sentence  2:  1.09656494770233


In [57]:
#word movers distance
#train data
from pyemd import emd
dtt=[]
a_wmd=[]
for i in range(0,750):
    dt= word_vectors.wmdistance(train_text1[i], train_text2[i])
    dtt = 1-dt # changing distance to similarity
    a_wmd.append(dtt)
print(a_wmd)
len(a_wmd)



[0.8884309001514052, 0.5741257532003886, 0.5320323360488237, 0.8555067159624765, 0.49444829899349385, 0.4549806926134433, 0.6222593501661784, 0.5562165519519189, 0.11853642267378728, 0.4166360632926469, 0.734512238136811, 0.13171365356245424, 0.4665243523944276, 0.734819310033962, 0.7714012022701, 0.8045313497769779, 0.5145932986174919, 0.8201260006626625, 0.8392093619846727, 0.5416500090808338, 0.6318603474412334, 0.8130532017421012, 0.7695813375003095, 0.6783038229190039, 0.8869323175945284, 0.76689392412847, 0.3791020078996137, 0.6652501212242841, 0.8707939766168266, 0.9348541384384188, 0.9065523170415969, 1.0, 0.5692595186770822, 0.5363184776280969, 0.32854506577306397, 0.7784939173110424, 0.25369500711504334, 0.8798658413154239, 0.4873269618684638, 0.5915320637662529, 0.6532400037980867, 0.31417796535069187, 0.6839285657081688, 0.07346025099925557, 0.386060361709602, 0.03681374279063421, 0.48726111674589645, 0.5816533449364882, 0.7389971711959278, 0.7770743611985673, 0.62694430201

750

In [58]:

# wmd for test data
from pyemd import emd
dtt1=[]
a_wmd1=[]
for i in range(0,318):
    dt= word_vectors.wmdistance(test_text1[i], test_text2[i])
    dtt1 = 1-dt # changing distance to similarity
    a_wmd1.append(dtt1)
print(a_wmd1)
len(a_wmd1)

[0.7072900741638799, 0.31153212143293096, 0.5864220102457904, 0.6340703673199739, 0.22827465626133425, 0.8914170875301811, 0.6170826530064678, 0.7758890107099998, 0.6279492977539278, 0.9233679560241389, 0.17680678711349462, 0.11772748411658107, 0.8628287819720942, 0.7108752902991742, 0.3920084170301994, 0.8096090136537744, 0.8051854651153241, 0.6570956382913617, 0.6337085199727195, 0.27159159256584153, 0.4677325385717406, 0.4485337048851703, 0.4866688214669841, 0.7245667362691375, 0.22370948615683595, 0.647660907850006, 0.7229751820471968, 0.31449213749926686, 0.48359718887694725, 0.752723957519607, 0.7966672003364076, 0.628560546699489, 0.8594476522719134, 0.6667115333846606, 0.4319702449331648, 0.7403356232238001, 0.5408148516443744, 0.8374873767551966, 0.670932348195205, 0.47864753334415744, 0.6880421169903799, 0.8391139568699335, 0.5919064771534475, 0.671668980269912, 0.8772357695317626, 0.7492432537353031, 0.0571708906541355, 0.5365477157422534, 0.7980379860031669, 0.7409930648003

318

In [59]:
# pearson Correlation 

import numpy as np
from scipy.stats import pearsonr    
b_pc, _ = pearsonr(a_wmd, train_labels)
print('Pearsons correlation for Train: %.5f' % b_pc)

Pearsons correlation for Train: 0.68271


In [60]:

from scipy.stats import pearsonr
b_pc_t, _ = pearsonr(a_wmd1, test_labels)
print('Pearsons correlation for Test: %.5f' % b_pc_t)

Pearsons correlation for Test: 0.69089


In [61]:
#storing variables
%store b_mse1
%store b_mse2
%store a_wmd
%store a_wmd1
%store b_pc
%store b_pc_t

Stored 'b_mse1' (float64)
Stored 'b_mse2' (float64)
Stored 'a_wmd' (list)
Stored 'a_wmd1' (list)
Stored 'b_pc' (float64)
Stored 'b_pc_t' (float64)
